In [10]:
import multiprocessing
import os
import re
import typing as T

from client.spotify import SpotipyClient
from client.yt_music import YTMusicClient
from config import (ROOT_DIR, SPOTIPY_CLIENT_ID, SPOTIPY_CLIENT_SECRET,
                    SPOTIPY_USER, YT_PLAYLISTS, YTMUSIC_OAUTH_HEADER_FILE)
from helpers.media import YTVideo
from helpers.utils import set_thumbnail
from pytube import Playlist

RAW_DIR = f"{ROOT_DIR}/raw"


def initialize_clients() -> tuple[SpotipyClient, YTMusicClient]:
    """Initializes Spotify and Youtube Music clients.

    Returns:
        A tuple containing the initialized SpotipyClient and YTMusicClient instances.
    """

    return (
        SpotipyClient(
            client_id=SPOTIPY_CLIENT_ID,
            client_secret=SPOTIPY_CLIENT_SECRET,
            user=SPOTIPY_USER,
        ),
        YTMusicClient(auth=YTMUSIC_OAUTH_HEADER_FILE),
    )


def download_song(yt_video: YTVideo, playlist):
    final_directory = f"{ROOT_DIR}/playlists/{playlist}"
    file_path = yt_video.download_mp3(
        raw_directory=RAW_DIR, final_directory=final_directory
    )
    set_thumbnail(file_path, yt_video.youtube.thumbnail_url)


def get_spotify_videos(sp_playlists, ytm_client) -> T.Dict[str, T.List[YTVideo]]:
    playlist_map = {}
    for playlist, songs in sp_playlists.items():
        videos = []
        for song in songs:
            title, artist = song
            track, album, video_id = ytm_client.return_top_video_match(title)
            video = YTVideo(
                video_id=video_id,
                track=track,
                artist=artist,
                album=album,
                playlist=playlist,
                comments="",
            )
            videos.append(video)
        playlist_map[playlist] = videos
    return playlist_map


def get_ytm_videos(playlists: T.Dict[str, str]) -> T.Dict[str, T.List[YTVideo]]:
    playlist_map = {}
    for playlist, url in playlists.items():
        videos = []
        for v in Playlist(url).videos:
            video_info = get_song_title_artist(v.title)
            video_id = v.video_id
            track = video_info["title"]
            artist = video_info["artist"]
            album = ""
            video = YTVideo(
                video_id=video_id,
                track=track,
                artist=artist,
                album=album,
                playlist=playlist,
                comments="",
            )
            videos.append(video)
        playlist_map[playlist] = videos
    return playlist_map


def get_song_title_artist(text):
    """
    This function attempts to extract the song title and artist from a string.

    Args:
      text: The string containing the song information.

    Returns:
      A dictionary containing the extracted song title and artist,
      or None if no song information is found.
    """
    # Use regular expressions to extract artist and title in various formats
    matches = re.search(r"(?P<artist>.*?)\s*-\s*(?P<title>.*)", text)
    if matches:
        return matches.groupdict()
    matches = re.search(r"(?P<title>.*?)\s*\(\s*(?P<artist>.*?)\s*\)", text)
    if matches:
        return matches.groupdict()
    return {"title": text, "artist": ""}


def func(index: int):
    print(index)





In [15]:
tmp = YTMusicClient(auth='test.json')

In [16]:
tmp.return_top_video_match("heize undo")

('i_-kwMJDjq0', '없었던일로', 'Undo')

In [12]:
from ytmusicapi.setup import setup_browser

In [13]:
raw = """Accept:
*/*
Accept-Encoding:
gzip, deflate, br, zstd
Accept-Language:
en-US,en;q=0.9,ko;q=0.8
Authorization:
SAPISIDHASH 1712059859_dafb5e370012d0fba83b0fa917c310e629467092
Content-Length:
3189
Content-Type:
application/json
Cookie:
SID=g.a000iAhVrjqxxCKORnmPw0fwg40HEku_b2GQeNofV_kALZqRznnCFAsS4xVPHFhIWV8DLAaEbwACgYKAdUSAQASFQHGX2MikVidfZPZGrOszsOhyxkNZRoVAUF8yKrLFbMyqvpuvpsVqsCVEHi50076; __Secure-1PSID=g.a000iAhVrjqxxCKORnmPw0fwg40HEku_b2GQeNofV_kALZqRznnCgtRbqMB-X2hzg6WRlMgbqAACgYKAcYSAQASFQHGX2MilqDYUNdtABAnuFF0jU7QWBoVAUF8yKpX35KcAMVTRn2Y8fiAfo7F0076; __Secure-3PSID=g.a000iAhVrjqxxCKORnmPw0fwg40HEku_b2GQeNofV_kALZqRznnCt47vDeoalwk7DAkMq13gDwACgYKAQ4SAQASFQHGX2Mii1O7nA2Gf2HNKqXPck63lBoVAUF8yKqElRkB6PjeWTqJ1KwS9EIC0076; HSID=AfOHvdwyd4ocVcyUq; SSID=AqGPed3mIWLpzyPIk; APISID=egoZ5d0Zoi8Db4pJ/AS7xDOBU42JG-lq9v; SAPISID=pivRROyIavSRCEv7/AAsVavG00tzwIbaZ7; __Secure-1PAPISID=pivRROyIavSRCEv7/AAsVavG00tzwIbaZ7; __Secure-3PAPISID=pivRROyIavSRCEv7/AAsVavG00tzwIbaZ7; VISITOR_PRIVACY_METADATA=CgJLUhIEGgAgDw%3D%3D; LOGIN_INFO=AFmmF2swRQIhAPkBNLI-0R3nVaGza_Kvi3HmbH8tC9ADd6PDiyD90YjeAiAhcjOyGAwDEuWQe5sGhrDOSnl6nlErVLdHp3xzwHbYmA:QUQ3MjNmdzRtbHZrd0FiSzJjYlNZWmdXLWxqSVJqZ0VQcEhUM1BwNlFoMkkwdmRQNzFBTDFjTVFiWHlsQUZJekpVYmdCN3d4a3FnZ0JaVXMtVzMzLW1PaEtZQW5Zc1J3V1RyVWtkRzRsaHZmNXkwR3U3dFFsY04tMzVEdEVWVGVjYjVlMkRfRTFrX0toY0plbzZUWFdsaC1oVEJUSUdHVzhn; _gcl_au=1.1.911865.1711878206; NID=512=tnK4CA7V8rn8C2kkewT2l6dFzNYv2eWOpYpf9OGBoqI8HVdwlxRT_ApNkeH-Y3miXQEnnSA8kEZyCqhQlaAOYcX6VkoA_91nYZUSRphxJ9LwTjIfppIR8DwE4S6EPLXv32VLSTxFNcHjd_WjI60cBX_HrX_sr6jB-69080QRI5f1uFwlng_qS2RNeYQGBUnsaTXQxOCa6xw5zks; VISITOR_INFO1_LIVE=naAJD_2ABB0; VISITOR_PRIVACY_METADATA=CgJLUhIEGgAgDw%3D%3D; YSC=WQ6687AeJAE; PREF=f6=40000080&f7=4100&tz=Asia.Seoul&f4=4000000; __Secure-1PSIDTS=sidts-CjIB7F1E_PKpz29xKSXWvGI7aov4QpcEML2wxgZUuJPbf10OetQ6zddOZKriNWBnID-FexAA; __Secure-3PSIDTS=sidts-CjIB7F1E_PKpz29xKSXWvGI7aov4QpcEML2wxgZUuJPbf10OetQ6zddOZKriNWBnID-FexAA; SIDCC=AKEyXzVSoozfTPnvYDfcF_CKuxRWc6igiYJlCxDmYy5W0DU_PzPVCI_48m9YJgwx-FknfFWx1A; __Secure-1PSIDCC=AKEyXzUQ7i8emQpdXaezrxLycuVHA8-XVYytoIeYaTR06UiGjjTx5BJfZOUQbzG2qyhKRRPk-A; __Secure-3PSIDCC=AKEyXzX_ydiZGiJ3gl7zwBK9_61dairQx_S-9qrYIobCa_zzn0QbOOT6Pf3Mnj0BUqDhBnwotg
Origin:
https://music.youtube.com
Referer:
https://music.youtube.com/
Sec-Ch-Ua:
"Google Chrome";v="123", "Not:A-Brand";v="8", "Chromium";v="123"
Sec-Ch-Ua-Arch:
"arm"
Sec-Ch-Ua-Bitness:
"64"
Sec-Ch-Ua-Full-Version:
"123.0.6312.87"
Sec-Ch-Ua-Full-Version-List:
"Google Chrome";v="123.0.6312.87", "Not:A-Brand";v="8.0.0.0", "Chromium";v="123.0.6312.87"
Sec-Ch-Ua-Mobile:
?0
Sec-Ch-Ua-Model:
""
Sec-Ch-Ua-Platform:
"macOS"
Sec-Ch-Ua-Platform-Version:
"14.4.1"
Sec-Ch-Ua-Wow64:
?0
Sec-Fetch-Dest:
empty
Sec-Fetch-Mode:
same-origin
Sec-Fetch-Site:
same-origin
User-Agent:
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36
X-Client-Data:
CJC2yQEIpLbJAQipncoBCL7nygEIkqHLAQjymM0BCIagzQEIuMjNAQje7M0BCMSAzgEI2oTOAQi1hc4BCLeGzgEIxIbOAQjAt8oiGLOpygEY1+vNARiY9c0BGNP+zQEY2IbOAQ==
Decoded:
message ClientVariations {
  // Active client experiment variation IDs.
  repeated int32 variation_id = [3300112, 3300132, 3313321, 3322814, 3330194, 3361906, 3362822, 3367992, 3372638, 3375172, 3375706, 3375797, 3375927, 3375940, 72522688];
  // Active client experiment variation IDs that trigger server-side behavior.
  repeated int32 trigger_variation_id = [3314867, 3372503, 3373720, 3374931, 3375960];
}
X-Goog-Authuser:
0
X-Goog-Visitor-Id:
CgtuYUFKRF8yQUJCMCjOpq-wBjIKCgJLUhIEGgAgDw%3D%3D
X-Origin:
https://music.youtube.com
X-Youtube-Bootstrap-Logged-In:
true
X-Youtube-Client-Name:
67
X-Youtube-Client-Version:
1.20240327.01.01"""

In [14]:
setup_browser("test.json", headers_raw=raw)

'{"Accept": "*/*", "Accept-Encoding": "gzip, deflate, br, zstd", "Accept-Language": "en-US,en;q=0.9,ko;q=0.8", "Authorization": "SAPISIDHASH 1712059859_dafb5e370012d0fba83b0fa917c310e629467092", "Content-Length": "3189", "Content-Type": "application/json", "Cookie": "SID=g.a000iAhVrjqxxCKORnmPw0fwg40HEku_b2GQeNofV_kALZqRznnCFAsS4xVPHFhIWV8DLAaEbwACgYKAdUSAQASFQHGX2MikVidfZPZGrOszsOhyxkNZRoVAUF8yKrLFbMyqvpuvpsVqsCVEHi50076; __Secure-1PSID=g.a000iAhVrjqxxCKORnmPw0fwg40HEku_b2GQeNofV_kALZqRznnCgtRbqMB-X2hzg6WRlMgbqAACgYKAcYSAQASFQHGX2MilqDYUNdtABAnuFF0jU7QWBoVAUF8yKpX35KcAMVTRn2Y8fiAfo7F0076; __Secure-3PSID=g.a000iAhVrjqxxCKORnmPw0fwg40HEku_b2GQeNofV_kALZqRznnCt47vDeoalwk7DAkMq13gDwACgYKAQ4SAQASFQHGX2Mii1O7nA2Gf2HNKqXPck63lBoVAUF8yKqElRkB6PjeWTqJ1KwS9EIC0076; HSID=AfOHvdwyd4ocVcyUq; SSID=AqGPed3mIWLpzyPIk; APISID=egoZ5d0Zoi8Db4pJ/AS7xDOBU42JG-lq9v; SAPISID=pivRROyIavSRCEv7/AAsVavG00tzwIbaZ7; __Secure-1PAPISID=pivRROyIavSRCEv7/AAsVavG00tzwIbaZ7; __Secure-3PAPISID=pivRROyIavSRCEv7/AAsVavG00

In [2]:
sp_client, ytm_client = initialize_clients()

In [3]:
ytm_client.return_top_video_match("heize undo")

IndexError: list index out of range

In [ ]:
sp_playlists = sp_client.get_spotify_playlists_and_songs(fuzzy_name="rekordbox")
sp_videos = get_spotify_videos(sp_playlists, ytm_client)
yt_videos = get_ytm_videos(YT_PLAYLISTS)

In [ ]:


if __name__ == "__main__":
    tracks = main()
    manager = multiprocessing.Manager()
    processes = []
    for playlist, videos in tracks.items():
        for video in videos:
            p = multiprocessing.Process(
                target=download_song,
                args=(
                    video,
                    playlist,
                ),
            )
            processes.append(p)
            p.start()

    for process in processes:
        process.join()